In [1]:
import pandas as pd

import warnings

from select import select

warnings.filterwarnings('ignore')

In [2]:
path = "E:\\MIMIC_III\\mimic-iii-clinical-database-1.4\\"

In [3]:
# load Patients and Diagnoses_icd
patients = pd.read_csv(path + '\\patients.csv')
diagnoses = pd.read_csv(path + '\\diagnoses_icd.csv')

In [4]:
print(f"Patients shape is {patients.shape}")
print(f"Diagnoses shape is {diagnoses.shape}")
print()

# Check if ICD codes are numeric or not
icd_column_dtype = diagnoses['ICD9_CODE'].dtype
print("Checking ICD codes:")
print(f"\tICD codes column dtype is {icd_column_dtype}")

# Check if all ICD codes are numeric
if pd.api.types.is_numeric_dtype(diagnoses['ICD9_CODE']):
    print("\tAll ICD codes are numeric.")
else:
    print("\tICD codes are not entirely numeric. Attempting to clean and convert...")

    # Attempt to clean and convert to numeric, ignoring errors
    diagnoses['ICD9_CODE'] = pd.to_numeric(diagnoses['ICD9_CODE'], errors='coerce')

    # Report how many could not be converted
    null_count = diagnoses['ICD9_CODE'].isnull().sum()
    print(f"\tConverted ICD codes to numeric. {null_count} rows could not be converted and contain NaN.")


Patients shape is (46520, 8)
Diagnoses shape is (651047, 5)

Checking ICD codes:
	ICD codes column dtype is object
	ICD codes are not entirely numeric. Attempting to clean and convert...
	Converted ICD codes to numeric. 97296 rows could not be converted and contain NaN.


In [5]:
# checking why ICD9_CODE columns is Object
diagnoses['ICD9_CODE'].isna().sum()

97296

In [6]:
diagnoses_cleaned = diagnoses.dropna(subset=['ICD9_CODE'])
# Verify the result
print(f"Diagnoses shape after dropping missing ICD9_CODE: {diagnoses_cleaned.shape}")

Diagnoses shape after dropping missing ICD9_CODE: (553751, 5)


In [7]:
# Check if ICD codes are numeric or not
icd_column_dtype = diagnoses_cleaned['ICD9_CODE'].dtype
print("Checking ICD codes:")
print(f"\tICD codes column dtype is {icd_column_dtype}")

# Check if all ICD codes are numeric
if pd.api.types.is_numeric_dtype(diagnoses['ICD9_CODE']):
    print("\tAll ICD codes are numeric.")
else:
    print("\tICD codes are not entirely numeric. Attempting to clean and convert...")

    # Attempt to clean and convert to numeric, ignoring errors
    diagnoses['ICD9_CODE'] = pd.to_numeric(diagnoses['ICD9_CODE'], errors='coerce')

    # Report how many could not be converted
    null_count = diagnoses['ICD9_CODE'].isnull().sum()
    print(f"\tConverted ICD codes to numeric. {null_count} rows could not be converted and contain NaN.")


Checking ICD codes:
	ICD codes column dtype is float64
	All ICD codes are numeric.


In [8]:
# now lets extract sepsis patients ( sepsis , severe sepsis , septic shock)
sepsis_codes = {
    99591.0: 'Sepsis',
    99592.0: 'Severe Sepsis',
    78552.0: 'Septic Shock'
}

In [9]:
# Filter for sepsis-related diagnoses
sepsis_diagnoses = diagnoses_cleaned[diagnoses_cleaned['ICD9_CODE'].isin(sepsis_codes.keys())]


In [10]:
sepsis_diagnoses.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'], dtype='object')

In [11]:
sepsis_diagnoses.shape

(7770, 5)

In [12]:
# Merge with the patients table to get demographic details
sepsis_patients = sepsis_diagnoses.merge(patients, on='SUBJECT_ID', how='inner')

In [13]:
print(sepsis_patients.head())
sepsis_patients.shape

   ROW_ID_x  SUBJECT_ID  HADM_ID  SEQ_NUM  ICD9_CODE  ROW_ID_y GENDER  \
0      1547         117   164853     16.0    99592.0       108      F   
1      1604         124   138376      6.0    99592.0       114      M   
2       505          64   172056      3.0    99591.0        57      F   
3       679          85   112077     18.0    99591.0        77      M   
4       131          21   111970      2.0    78552.0        18      M   

                   DOB                  DOD             DOD_HOSP  \
0  2083-12-28 00:00:00  2133-12-01 00:00:00  2133-12-01 00:00:00   
1  2090-11-19 00:00:00  2166-02-01 00:00:00  2166-02-01 00:00:00   
2  2116-06-27 00:00:00                  NaN                  NaN   
3  2090-09-18 00:00:00  2167-09-12 00:00:00  2167-09-12 00:00:00   
4  2047-04-04 00:00:00  2135-02-08 00:00:00  2135-02-08 00:00:00   

               DOD_SSN  EXPIRE_FLAG  
0  2133-12-01 00:00:00            1  
1  2166-02-01 00:00:00            1  
2                  NaN            0  


(7770, 12)

In [14]:
print(len(diagnoses['ICD9_CODE'].unique()))

5954


## ----------------------------------------------------

## ----------------------------------------------------

# Now exclude patients with multiple admissions except for first admissions

In [21]:
admissions = pd.read_csv(path + '\\admissions.csv')

In [22]:
admissions.shape

(58976, 19)

In [23]:
admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [31]:
admissions['ADMITTIME'].isna().sum()

0

In [32]:
# Count the number of admissions per patient
admission_counts = admissions.groupby('SUBJECT_ID').size().reset_index(name='ADMISSION_COUNT')

print(admission_counts)


       SUBJECT_ID  ADMISSION_COUNT
0               2                1
1               3                1
2               4                1
3               5                1
4               6                1
...           ...              ...
46515       99985                1
46516       99991                1
46517       99992                1
46518       99995                1
46519       99999                1

[46520 rows x 2 columns]


In [33]:
# Filter out patients with multiple admissions
single_admissions_patients = admission_counts[admission_counts['ADMISSION_COUNT'] == 1]

print(single_admissions_patients)

       SUBJECT_ID  ADMISSION_COUNT
0               2                1
1               3                1
2               4                1
3               5                1
4               6                1
...           ...              ...
46515       99985                1
46516       99991                1
46517       99992                1
46518       99995                1
46519       99999                1

[38983 rows x 2 columns]


In [41]:
# choose sepsis patients from admissions
filtered_admissions = admissions[admissions['SUBJECT_ID'].isin(sepsis_patients['SUBJECT_ID'])]
filtered_admissions.shape

(8703, 19)

In [42]:
filtered_admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
19,40,38,185910,2166-08-10 00:28:00,2166-09-04 11:30:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,NaN,NaN,ACUTE MYOCARDIAL INFARCTION-SEPSIS,0,1
22,456,357,174486,2197-12-06 07:13:00,2198-01-03 14:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2197-12-06 06:28:00,2197-12-06 07:43:00,GASTROINTESTINAL BLEED,0,1
23,457,357,145674,2198-08-02 04:49:00,2198-10-26 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2198-08-02 02:03:00,2198-08-02 05:37:00,SEIZURE,0,1
24,458,357,122609,2198-11-01 22:36:00,2198-11-14 14:20:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2198-11-01 18:01:00,2198-11-01 23:06:00,SEPSIS,0,1
25,459,357,101651,2199-10-20 12:05:00,2199-10-23 17:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2199-10-20 09:34:00,2199-10-20 13:25:00,PNEUMONIA,0,1


In [44]:
filtered_admissions = filtered_admissions[['SUBJECT_ID', 'ADMITTIME']].dropna(subset=['ADMITTIME'])
filtered_admissions.head()

,SUBJECT_ID,ADMITTIME
19,38,2166-08-10 00:28:00
22,357,2197-12-06 07:13:00
23,357,2198-08-02 04:49:00
24,357,2198-11-01 22:36:00
25,357,2199-10-20 12:05:00


In [46]:
filtered_admissions = filtered_admissions.sort_values(by='ADMITTIME')
filtered_admissions.head()

,SUBJECT_ID,ADMITTIME
7378,4521,2100-06-28 19:29:00
37595,31585,2100-07-02 19:28:00
38886,42357,2100-07-14 02:04:00
50353,73131,2100-07-14 18:14:00
54560,84585,2100-08-03 00:26:00


In [47]:
filtered_admissions = filtered_admissions.drop_duplicates(subset='SUBJECT_ID', keep='first')
filtered_admissions.shape

(4689, 2)

In [48]:
sepsis_filtered = sepsis_patients[sepsis_patients['SUBJECT_ID'].isin(filtered_admissions['SUBJECT_ID'])]
sepsis_filtered.shape

(7770, 12)

In [87]:
# Convert DOB and ADMITTIME to datetime format
patients['DOB'] = pd.to_datetime(patients['DOB'], errors='coerce')
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'], errors='coerce')

# Merge patients and admissions to calculate age
merged_df = pd.merge(
    admissions[['SUBJECT_ID', 'ADMITTIME']],
    patients[['SUBJECT_ID', 'DOB']],
    on='SUBJECT_ID',
    how='inner'
)

# Calculate age in years
merged_df['AGE'] = (merged_df['ADMITTIME'] - merged_df['DOB']).dt.days // 365

# Filter out shifted DOBs (age > 300 years)
merged_df = merged_df[merged_df['AGE'] <= 300]

OverflowError: Overflow in int64 addition

In [88]:
patients.DOB.sample(10)

1891    2151-06-17
31865   2093-02-03
37084   1868-10-25
32469   2114-04-17
22018   2101-08-17
18838   2099-02-20
24329   2101-02-24
15701   2096-02-29
6579    2157-12-22
4652    2160-05-30
Name: DOB, dtype: datetime64[ns]

In [98]:
# Convert DOB to datetime format
patients['DOB'] = pd.to_datetime(patients['DOB'], errors='coerce')

# Identify shifted DOBs (year < 1900)
shifted_dobs = patients[patients['DOB'].dt.year < 1900]

# Count the number of shifted DOBs
shifted_count = shifted_dobs.shape[0]

# Display the result
print(f"Number of shifted DOB entries: {shifted_count}")

# Optionally, view the shifted rows
print(shifted_dobs[['SUBJECT_ID', 'DOB']].head())

Number of shifted DOB entries: 1959
     SUBJECT_ID        DOB
52          698 1864-11-16
61          709 1817-01-20
161         813 1879-12-15
174         827 1874-01-04
237          19 1808-08-05


In [99]:
shifted_dobs.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
52,658,698,F,1864-11-16,2168-04-22 00:00:00,NaN,2168-04-22 00:00:00,1
61,667,709,F,1817-01-20,2118-10-13 00:00:00,NaN,2118-10-13 00:00:00,1
161,767,813,M,1879-12-15,2180-01-30 00:00:00,2180-01-30 00:00:00,2180-01-30 00:00:00,1
174,780,827,F,1874-01-04,NaN,NaN,NaN,0
237,16,19,M,1808-08-05,2109-08-18 00:00:00,NaN,2109-08-18 00:00:00,1


In [124]:
# Convert DOB to datetime format
patients['DOB'] = pd.to_datetime(patients['DOB'], errors='coerce')

# Drop rows with shifted DOBs (year < 1900)
patients_cleaned = patients[patients['DOB'].dt.year >= 1900]

# Display the number of remaining rows
print(f"Number of rows after dropping shifted DOBs: {patients_cleaned.shape[0]}")


Number of rows after dropping shifted DOBs: 44561


In [143]:
# Convert DOB and ADMITTIME to datetime format
sepsis_patients['DOB'] = pd.to_datetime(sepsis_patients['DOB'], errors='coerce')
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'], errors='coerce')

# Filter out shifted DOBs (year < 1900)
patients_cleaned = sepsis_patients[sepsis_patients['DOB'].dt.year >= 1900]

# Merge PATIENTS (cleaned) and ADMISSIONS on SUBJECT_ID
# merged_df = pd.merge(
#     patients_cleaned[['SUBJECT_ID', 'DOB']],
#     admissions[['SUBJECT_ID', 'ADMITTIME']],
#     on='SUBJECT_ID',
#     how='inner'
# )
# 
# # Calculate age at the time of admission
# merged_df['AGE'] = (merged_df['ADMITTIME'] - merged_df['DOB']).dt.days // 365
# 
# # Display the resulting DataFrame with calculated ages
# print(merged_df[['SUBJECT_ID', 'DOB', 'ADMITTIME', 'AGE']].head())


In [126]:
merged_df.shape

(56401, 3)

In [127]:
# Identify extreme DOB or ADMITTIME values
#invalid_dobs = patients_cleaned[~patients_cleaned['DOB'].between(pd.Timestamp('1900-01-01'), pd.Timestamp('2300-01-01'))]
invalid_admittimes = admissions[~admissions['ADMITTIME'].between(pd.Timestamp('1900-01-01'), pd.Timestamp('2300-01-01'))]

#print(f"Invalid DOBs:\n{invalid_dobs}")
print(f"Invalid ADMITTIMEs:\n{invalid_admittimes}")


Invalid ADMITTIMEs:
Empty DataFrame
Columns: [ROW_ID, SUBJECT_ID, HADM_ID, ADMITTIME, DISCHTIME, DEATHTIME, ADMISSION_TYPE, ADMISSION_LOCATION, DISCHARGE_LOCATION, INSURANCE, LANGUAGE, RELIGION, MARITAL_STATUS, ETHNICITY, EDREGTIME, EDOUTTIME, DIAGNOSIS, HOSPITAL_EXPIRE_FLAG, HAS_CHARTEVENTS_DATA]
Index: []


In [144]:
patients_cleaned['DOB'] = patients_cleaned['DOB'].dt.to_pydatetime()
admissions['ADMITTIME'] = admissions['ADMITTIME'].dt.to_pydatetime()

In [145]:
# Merge PATIENTS (cleaned) and ADMISSIONS (cleaned)
merged_df = pd.merge(
    patients_cleaned[['SUBJECT_ID', 'DOB']],
    admissions[['SUBJECT_ID', 'ADMITTIME']],
    on='SUBJECT_ID',
    how='inner'
)

# Calculate age using native Python datetime
merged_df['AGE'] = merged_df.apply(
    lambda row: (row['ADMITTIME'].to_pydatetime() - row['DOB'].to_pydatetime()).days // 365 if row['DOB'] and row['ADMITTIME'] else None,
    axis=1
)

# Display results
print(merged_df[['SUBJECT_ID', 'DOB', 'ADMITTIME', 'AGE']].head())

   SUBJECT_ID        DOB           ADMITTIME  AGE
0         117 2083-12-28 2133-04-07 16:29:00   49
1         117 2083-12-28 2133-11-13 21:34:00   49
2         124 2090-11-19 2161-12-17 03:39:00   71
3         124 2090-11-19 2165-05-21 21:02:00   74
4         124 2090-11-19 2165-12-31 18:55:00   75


In [112]:
sepsis_patients['DOB'] = sepsis_patients['DOB'].dt.to_pydatetime()
# Filter rows where the year in DOB is 1872
dob_1872 = sepsis_patients[sepsis_patients['DOB'].dt.year == 1872]

# Display the filtered rows
print(dob_1872)

# Optionally, count the rows with DOB in 1872
print(f"Number of rows with DOB in 1872: {dob_1872.shape[0]}")


AttributeError: Can only use .dt accessor with datetimelike values

In [115]:
sepsis_patients.DOB.sample(10)

1631    2063-05-18 00:00:00
7468    1884-07-30 00:00:00
689     2107-05-12 00:00:00
2388    2160-07-16 00:00:00
1770    2045-08-27 00:00:00
5642    2084-11-27 00:00:00
462     2111-07-10 00:00:00
2811    2102-03-14 00:00:00
2260    2130-08-04 00:00:00
5470    2134-09-18 00:00:00
Name: DOB, dtype: object

In [116]:
# Save the sepsis_patients DataFrame as a CSV file
sepsis_patients.to_csv("sepsis_patients.csv", index=False)

print("sepsis_patients has been saved as 'sepsis_patients.csv'.")


sepsis_patients has been saved as 'sepsis_patients.csv'.


In [132]:
merged_df.shape

(56401, 4)

In [136]:
# Merge with sepsis_patients to add AGE
sepsis_with_age = pd.merge(
    sepsis_patients,
    merged_df[['SUBJECT_ID', 'AGE']],
    on='SUBJECT_ID',
    how='left'
)

sepsis_with_age.shape

(16858, 13)

In [137]:
# Filter merged_df to include only SUBJECT_IDs present in sepsis_patients
filtered_merged_df = merged_df[merged_df['SUBJECT_ID'].isin(sepsis_patients['SUBJECT_ID'])]

# Display the filtered DataFrame
print(filtered_merged_df)


       SUBJECT_ID        DOB           ADMITTIME  AGE
3             250 2164-12-27 2188-11-12 09:22:00   23
24            269 2130-09-30 2170-11-05 11:04:00   40
54            689 2128-12-27 2180-04-03 15:22:00   51
55            689 2128-12-27 2182-03-18 12:09:00   53
56            689 2128-12-27 2183-04-01 13:56:00   54
...           ...        ...                 ...  ...
56372       44018 2067-06-11 2155-10-21 02:05:00   88
56376       44036 2132-05-07 2186-08-22 03:23:00   54
56384       44064 2115-11-18 2200-02-23 22:35:00   84
56385       44064 2115-11-18 2200-04-21 00:48:00   84
56386       44064 2115-11-18 2200-04-27 15:28:00   84

[8113 rows x 4 columns]


In [141]:
merged_df.AGE.sample(10)

7464     58
35511    39
15725    67
17114     0
15518    44
15912     0
51154    58
393      60
13663     0
16939    83
Name: AGE, dtype: int64

In [142]:
# Filter merged_df to include only patients under 18
under_18_df = merged_df[merged_df['AGE'] < 18]

# Display the filtered DataFrame
print(under_18_df)

# Count the number of patients under 18
print(f"Number of patients under 18: {under_18_df.shape[0]}")


       SUBJECT_ID        DOB           ADMITTIME  AGE
14            258 2124-09-19 2124-09-19 03:59:00    0
15            260 2105-03-23 2105-03-23 10:23:00    0
19            264 2162-11-30 2162-11-30 02:19:00    0
41            675 2158-04-05 2158-04-05 08:54:00    0
43            677 2195-08-12 2195-08-12 15:36:00    0
...           ...        ...                 ...  ...
54914       31826 2198-01-20 2198-01-20 07:38:00    0
54916       31828 2163-04-04 2163-04-04 11:18:00    0
55941       84137 2140-07-25 2157-10-16 19:21:00   17
55963       84227 2151-03-11 2168-03-19 15:14:00   17
56005       84402 2174-04-27 2192-02-15 21:59:00   17

[8210 rows x 4 columns]
Number of patients under 18: 8210


In [146]:
merged_df.shape

(16267, 4)